# Dataset

*** 들어가기전 ***

파이토치 설치 : 구글 파이토치 검색

In [2]:
import os
import numpy as np

import torch
import torch.nn as nn

*** lst_label = [f for f in lst_data if f.startswith('label')] ***

startswith를 이용해서 prefixed 되어있는 리스트만 따로 정리할 수 있다. 
for 문 앞에 문자를 쓰면 바로 실행 가능하다. 

EX) s.startswitch('i like') #i like로 시작하는 문자열인가?
리턴 : True, false값


*** 뉴럴넷에 들어가는 모든 Input은 3개의 축이 들어가야 한다.(파이토치에 들어가는 디멘젼은 반드시 3개여야 한다.) 채널이 없는 경우 혹은 채널의 축이 무시되는 경우는 임의로 채널을 생성해야 한다.***

REF) subplot은 2 by 2 디멘젼만 받기 때문에 3개의 축을 만들면 에러가 뜬다.


***  data = {'input' : input, 'label' : label} ***
사전형 자료

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        
        lst_data = os.listdir(self.data_dir)
        
        lst_label = [f for f in lst_data if f.startswitch('label')]
        lst_input = [f for f in lst_data if f.startswitch('input')]
        
        lst_label.sort()
        lst_input.sort()
        
        self.lst_label = lst_label
        self.lst_input = lst_input
        
    def __len__(self):
        return len(self.lst_label)
    
    def __getitem__(self,index):
        label = np.load(os.path.join(self.data_dir, self.lst_label[index]))
        input = np.load(os.path.join(slef.data_dir, self.lst_input[index]))
        
        label = label/255.0
        input = input/255.0
        
        if label.ndim == 2 :
            label = label[:, :, np.newaxis]
        if input.ndim ==2:
            input = input[:, :, np.newaxis]
            
        data = {'input' : input, 'label' : label}
        
        if self.transform:
            data = self.transform(data)
            
        return data

SyntaxError: unexpected EOF while parsing (<ipython-input-3-16086400e8dd>, line 3)

*** Transpose 구현하기 ***

numpy image의 차원은 (y, x, channel)로 구성되어 있으며 파이토치의 input으로 들어가기 위해서 필요한 차원은 (channel, Y, X)이다.
numpy image의 채널을 첫 번재로 옮기고 나머지 축을 그대로 두기 위해서 label.transpose((2,0,1).astype(np.float32))를 해준다.

numpy를 텐서로 넘겨주기 위해서 torch.from_numpy()라는 함수를 이용한다.


*** input = (input - self.mean) / self.std ***

라벨은 0 또는 1이라는 클래스로 정의되어 있기 때문에 하면 Normalize를 하면 안되고 이미지에만 적용한다.

(본 예제에선 Normailzation이라고 말했지만 실제론 Standardization이다. (standard scaler, z-score normalization) 기존 데이터를 평균이 0, 표준편차가 1인 데이터로 만들며 데이터의 분포가 가우시안 분포일 때 유용하다. 표준편차로 나누고 평균을 뺀다.)

In [ ]:
## 트렌스 폼 구현하기
class ToTensor(object):
    def __call__(self, data):
        label, input = data['label'], data['input']
        
        label = label.transpose((2, 0, 1)).astype(np.float32)
        input = input.transpose((2, 0, 1)).astype(np.float32)
        
        data - {'label' : torch.from_numpy(label), 'input' : torch.from_numpy(input)}
        
        return data
    
class Normalization(object):
    def __init__(self, mean=0.5, std=0.5):
        self.mean = mean
        self.std = std
        
    def __call__(self,data):
        label, input = data['label']j, data['input']
        
        input = (input - self.mean) / self.std
        
        return data
    
class RandomFlip(object):
    def __call__(self, data):
        label, input = data['label'], datea['input']
        
        if np.random.rand() > 0.5
            label = np.fliplr(label)
            input = np.fliplr(input)
            
        if np.random.rand() > 0.5
            label = np.fliqud(label)
            input = np.fliqud(input)
            
        data = {'label' : label, 'input' : input}
        
        return data
        
        